## Collect labeled trips from the database and expand user inputs.
You need to have the database running before running this notebook.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from uuid import UUID

import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/mallen2/alternate_branches/eval-compatible-server/e-mission-server')

import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.trip_queries as esdtq
import emission.core.wrapper.user as ecwu

import confusion_matrix_handling as cm_handling
from confusion_matrix_handling import MODE_MAPPING_DICT
import get_EC
import helper_functions as hf

import sklearn.model_selection as skm

from sklearn.model_selection import KFold
from sklearn import linear_model

import scipy

METERS_TO_MILES = 0.000621371 # 1 meter = 0.000621371 miles

df_EI = pd.read_csv(r'Public_Dashboard/auxiliary_files/energy_intensity.csv') # r stands for raw string, only matters if the path is on Windows

In [ ]:
import emission.core.get_database as edb

chosen_program = 'all'
all_user_list = []
programs_all = {}
for u in edb.get_uuid_db().find():         # add users to proper locations in programs 
    program = u["user_email"].split("_")[0]    # This info is in the Stage_uuids collection of the database
    uuid = u["uuid"]
    if program not in programs_all.keys(): programs_all[program] = []
    programs_all[program].append(uuid)
    all_user_list.append(uuid)

user_list = programs_all[chosen_program] if chosen_program is not 'all' else all_user_list
os_map = {}

for u in user_list:
    profile = ecwu.User(u).getProfile()
    if 'curr_platform' in profile:
        os_map[u] = profile['curr_platform']
    else:
        print("Removed a user who had no OS information.")
        user_list.remove(u) # Note: this removes u from programs_all[chosen_program] as well.
        no_os_user = u

# Split UUIDs by program
# ue stands for user_email
uuid_program_map = {}
for ue in edb.get_uuid_db().find():
    program = ue['user_email'].split("_")[0]
    uuid = ue['uuid']
    uuid_program_map[uuid] = program

print('Number of participants in each program:')
print({program: len(programs_all[program]) for program in programs_all})

In [ ]:
# Takes ~ 2 min 45 s on Macbook Pro for all ceo data up to May 2022.
expanded_labeled_trips = hf.get_expanded_labeled_trips(user_list)
expanded_labeled_trips['os'] = expanded_labeled_trips.user_id.map(os_map)
expanded_labeled_trips['program'] = expanded_labeled_trips['user_id'].map(uuid_program_map)

expanded_labeled_trips = expanded_labeled_trips.drop(labels = ['source', 'end_fmt_time', 'end_loc', 'raw_trip',
    'start_fmt_time', 'start_loc','start_local_dt_year', 'start_local_dt_month', 'start_local_dt_day',
    'start_local_dt_hour', 'start_local_dt_minute', 'start_local_dt_second',
    'start_local_dt_weekday', 'start_local_dt_timezone',
    'end_local_dt_year', 'end_local_dt_month', 'end_local_dt_day',
    'end_local_dt_hour', 'end_local_dt_minute', 'end_local_dt_second',
    'end_local_dt_weekday', 'end_local_dt_timezone'], axis = 1)

In [ ]:
%store expanded_labeled_trips